<a href="https://colab.research.google.com/github/JorgeVitor30/logicomp-projeto/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running All pip install necessary


In [19]:
!pip install python-sat
!pip install pysat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing All functions and formulas

In [27]:
from formula import *
from functions import *

from pysat.solvers import Glucose3, Solver
from pysat.formula import IDPool



## Inicitialization Solver

In [47]:
g = Glucose3()

## Getting All values (slots, courses, common_courses)

In [41]:
slots = int(get_count_slots())
courses = get_all_courses()
common_inscriptions_courses = get_time_pairs()

## Applying all restrictions

In [49]:
for x in range(0, len(common_inscriptions_courses)):
    if x == 0:

        common_inscriptions_courses_restrictions = turn_common_inscriptions_pair_minicourses_to_propositional_logic(common_inscriptions_courses[0][0], common_inscriptions_courses[0][1], slots)
    else:
        common_inscriptions_courses_restrictions = And(common_inscriptions_courses_restrictions, turn_common_inscriptions_pair_minicourses_to_propositional_logic(common_inscriptions_courses[0][0], common_inscriptions_courses[0][1], slots))
    


for x in range(0, len(courses)):
    if x == 0:
        at_least_in_one_slot_restrictions = at_least_in_one_slot(x + 1, slots)
        max_in_one_slot_restrictions = max_in_one_slot(x + 1, slots)
    else:
        at_least_in_one_slot_restrictions = And(at_least_in_one_slot_restrictions, at_least_in_one_slot(x + 1, slots))
        max_in_one_slot_restrictions = And(max_in_one_slot_restrictions, max_in_one_slot(x + 1, slots))

all_restrictions = And(And(max_in_one_slot_restrictions, at_least_in_one_slot_restrictions), common_inscriptions_courses_restrictions)

## Transforming to CNF form

In [45]:
cnf_restrictions = cnf(all_restrictions)


clause_form_restrictions =  parse_cnf_to_clause_form(cnf_restrictions)

id_clauses = parse_clauses_to_id(clause_form_restrictions)


clause_set = []
id_clause_set = []

for clause in clause_form_restrictions:
    for literal in clause:
        if literal not in clause_set:
            clause_set.append(literal)

for clause in id_clauses:
    for literal in clause:
        if literal not in id_clause_set:
            id_clause_set.append(literal)
        

print(clause_set)
print(id_clause_set)

['¬x_1_1', '¬x_1_2', '¬x_1_3', '¬x_2_1', '¬x_2_2', '¬x_2_3', '¬x_3_1', '¬x_3_2', '¬x_3_3', '¬x_4_1', '¬x_4_2', '¬x_4_3', 'x_1_1', 'x_1_2', 'x_1_3', 'x_2_1', 'x_2_2', 'x_2_3', 'x_3_1', 'x_3_2', 'x_3_3', 'x_4_1', 'x_4_2', 'x_4_3']
[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


## Verify Satisfability and getting Valoration

In [50]:
g.append_formula(id_clauses)
g.solve()
valoration =  g.get_model()


output = []

print(valoration)
for id in valoration:
    if id > 0:
        output.append(clause_set[id_clause_set.index(id)])

print(output)

for course_variable in output:
    print(f"O curso {course_variable[2]} ficará no horário {course_variable[4]}")


[-1, 2, -3, 4, -5, -6, 7, -8, -9, 10, -11, -12]
['x_1_2', 'x_2_1', 'x_3_1', 'x_4_1']
O curso 1 ficará no horário 2
O curso 2 ficará no horário 1
O curso 3 ficará no horário 1
O curso 4 ficará no horário 1
